In [7]:
from numpy import array,float64
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import LSTM,RepeatVector,TimeDistributed
from keras.layers import Dropout
import os

In [ ]:
path = os.path.dirname(os.getcwd())

df_train = load_dataset(path + '/data/2015_data.csv')
df_test = load_dataset(path + '/data/2015_test.csv')

normalize_dataframes(df_train,df_test)

train_set = get_df_values(df_train) 
test_set = get_df_values(df_test)

X,y = split_sequences(sequences=train_set, n_steps_in = 60, n_steps_out = 5)

model = compile_model(n_steps_in = 60,n_steps_out = 5,n_features_in = 51)
print(model.summary())

history = train_model(model=model,X=X,y=y,_validation_split = 0.3,_epoch=100,_batch_size=32,
_modelCheckpoint_path= path + '/MPI/TempNormalized',_modelCheckpoint_monitor='val_accuracy')


In [2]:
def load_dataset(name):
    df = read_csv(name)
    df.drop(['Timestamp','Status'],axis=1, inplace=True)
    return df

In [3]:
def normalize_dataframes(df_train,df_test):
    sc = MinMaxScaler(feature_range = (0, 1))

    df_train.loc[:, df_train.dtypes == float64] = sc.fit_transform(df_train.loc[:, df_train.dtypes == float64])
    df_test.loc[:, df_test.dtypes == float64] = sc.transform(df_test.loc[:, df_test.dtypes == float64])


In [4]:
def get_df_values(df):
    return df.values

In [5]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [10]:
def compile_model(n_steps_in,n_steps_out,n_features_in):
    # define model
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_steps_in, n_features_in)))
    model.add(RepeatVector(n_steps_out))
    model.add(LSTM(200, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(51)))
    model.compile(optimizer='adam', loss='mse')

    return model

# fit model



In [6]:
def train_model(model,X,y,_validation_split,_epoch,_batch_size,_modelCheckpoint_path,_modelCheckpoint_monitor):
    return model.fit(X,y,validation_split=_validation_split, epochs = _epoch, batch_size = _batch_size, callbacks= [EarlyStopping(), ModelCheckpoint(_modelCheckpoint_path, monitor=_modelCheckpoint_monitor,save_best_only=True)], verbose=1)